In [1]:
!python -V

Python 3.8.10


In [2]:
import pandas as pd
import numpy as np

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


In [8]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df.duration.mean()

19.167224093791006

In [10]:
df.isnull().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID               958267
DOlocationID               162220
SR_Flag                   1154112
Affiliated_base_number        885
duration                        0
dtype: int64

In [11]:
missing_pickup = df.PUlocationID.isnull().sum()/df.shape[0]
missing_pickup

0.8303067639882438

#### remove duration outliers

In [12]:
 df = df[(df.duration >= 1) & (df.duration <= 60)]

In [13]:
df.shape[0]

1109826

In [14]:
missing_pickup = df.PUlocationID.isnull().sum()/df.shape[0]
missing_pickup

0.8352732770722617

In [15]:
df.PUlocationID.fillna(value=-1, inplace=True)
df.DOlocationID.fillna(value=-1, inplace=True)


In [16]:
df.PUlocationID.unique().shape

(262,)

In [17]:
df.DOlocationID.unique().shape

(263,)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  float64       
 4   DOlocationID            1109826 non-null  float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 76.2+ MB


In [19]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']

In [20]:
df[categorical] = df[categorical].astype(str)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 76.2+ MB


In [22]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107212091

In [23]:
X_train.shape

(1109826, 525)

#### evalutaion on Febuary data

In [24]:
df_feb = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [25]:
df_feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037692 entries, 0 to 1037691
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1037692 non-null  object        
 1   pickup_datetime         1037692 non-null  datetime64[ns]
 2   dropOff_datetime        1037692 non-null  datetime64[ns]
 3   PUlocationID            153001 non-null   float64       
 4   DOlocationID            885340 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1037692 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 55.4+ MB


In [26]:
df_feb['duration'] = df_feb.dropOff_datetime - df_feb.pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)

In [27]:
 df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [28]:
df_feb.PUlocationID.fillna(value=-1, inplace=True)
df_feb.DOlocationID.fillna(value=-1, inplace=True)


In [29]:
df_feb[categorical] = df_feb[categorical].astype(str)

In [30]:
df_feb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990113 entries, 1 to 1037691
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   dispatching_base_num    990113 non-null  object        
 1   pickup_datetime         990113 non-null  datetime64[ns]
 2   dropOff_datetime        990113 non-null  datetime64[ns]
 3   PUlocationID            990113 non-null  object        
 4   DOlocationID            990113 non-null  object        
 5   SR_Flag                 0 non-null       object        
 6   Affiliated_base_number  990113 non-null  object        
 7   duration                990113 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 68.0+ MB


#### find the DOlocationID that was in not in the january data and replace it with a -1 for the febuary data

In [31]:
jan_DO = df.DOlocationID.unique()
feb_DO = df_feb.DOlocationID.unique()
np.setdiff1d(feb_DO,jan_DO)

array(['110.0'], dtype=object)

In [32]:
df_feb.loc[df_feb.DOlocationID=='110.0']

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
144951,B03060,2021-02-05 13:18:45,2021-02-05 13:58:48,-1.0,110.0,None,B03060,40.050000
891678,B00972,2021-02-25 06:51:37,2021-02-25 06:54:21,176.0,110.0,None,B00972,2.733333
892044,B01452,2021-02-25 06:51:37,2021-02-25 06:54:21,176.0,110.0,None,B01452,2.733333


#### drop the DOlocationID the trained model has never seen from the validation data

In [33]:
df_feb.drop(df_feb[df_feb.DOlocationID=='110.0'].index, inplace=True)

In [34]:
df_feb.loc[df_feb.DOlocationID=='110.0']

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration


In [35]:
df_feb.shape

(990110, 8)

In [36]:
val_dicts = df_feb[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_val = dv.fit_transform(val_dicts)

In [37]:
target = 'duration'
y_val = df_feb[target].values

In [38]:
y_pred = lr.predict(X_val)

In [39]:
mean_squared_error(y_val, y_pred, squared=False)

11.014253833370951